# 01-09 : Aspect Based Sentiment Analysis (LLM)

In [1]:
import os
import pandas as pd
import json
from time import sleep
from typing import List, Dict, Tuple
from pprint import pprint
from tqdm.notebook import tqdm

from dotenv import load_dotenv, find_dotenv
import openai
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

In [4]:
# read local .env file
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ['OPENAI_API_KEY']

In [5]:
tqdm.pandas()

## Data Load

In [7]:
# load the human feedback dataset
df_source = pd.read_parquet('../../data/interim/01-06_human_classified.parquet')

# show the data loaded
print(df_source.shape)
display(df_source.head(3))

(235, 17)


,id,created_at,review_rating,review_title,review_content,business_slug,chatbot_related,chatbot_evidence,chatbot_classification,chatbot_description,chatbot_suggestion,complaint_classification,complaint_service,complaint_description,complaint_suggestion,human_chatbot_classification,human_complaint_classification
0,3344640,2021-01-07 13:22:34,1,No option to speak to the agent on the custome...,Am not able to call vodacom to block my number...,vodacom,1,The customer mentioned that the Tobi bot canno...,Customer care assistance,The customer is complaining about not being ab...,Improve the chatbot's availability and provide...,Customer care,Vodacom,The customer is unable to call Vodacom to bloc...,Provide an option for customers to speak to an...,"limited functionality, unable to contact human...",blacklist
1,3347241,2021-01-10 11:32:59,1,"Airtime charged, but not credited to my phone",Bought Airtime online through the Vodacom App ...,vodacom,1,The customer mentioned trying to chat with TOB...,Customer service,The customer complained about being thrown out...,Improve the stability of the chatbot to preven...,Billing,Airtime,The customer bought airtime online but it was ...,Investigate the issue and credit the airtime t...,"technical error, unable to contact human agent",missing airtime
2,3353838,2021-01-15 11:32:11,1,Chatbot Tobi/ Voice Bundle,I am disappointed at how your service has beco...,vodacom,1,The complaint mentions the introduction of a c...,Limited functionality,The chatbot has made it impossible for custome...,Improve the chatbot's capabilities to handle a...,Service issue,Voice Bundle,Failed to load voice bundle but debited the cu...,Load the customer's voice bundle or reimburse ...,"limited functionality, unable to contact human...",voice bundle


## Aspects

In [8]:
entities_aspects = {
    "Billing & Payments": [
        "Billing accuracy",
        "Payment methods",
        "Refunds/credits",
        "Hidden charges",
        "Monthly costs"
    ],
    "Network & Connectivity": [
        "Signal strength",
        "Network coverage",
        "Data speeds (4G, 5G, etc.)",
        "Call quality",
        "Dropped calls",
        "Roaming"
    ],
    "Customer Service": [
        "Responsiveness",
        "Friendliness/professionalism",
        "Knowledge/competence",
        "Resolution time",
        "Availability (e.g., 24/7 support)"
    ],
    "Chatbots": [
        "User-friendliness",
        "Response accuracy",
        "Speed of response",
        "Ability to understand query",
        "Escalation to human agents"
    ],
    "Account & Plans": [
        "Account management (online portal/apps)",
        "Plan flexibility",
        "Plan pricing",
        "Upgrade/downgrade process",
        "Promotions and offers"
    ],
    "Hardware/Devices": [
        "Setup/ease of installation",
        "Device reliability",
        "Device performance/speed",
        "Rental vs. purchase options",
        "Technical issues"
    ],
    "Value-added Services": [
        "Quality of service",
        "Pricing/value for money",
        "Reliability",
        "Content variety (for streaming)",
        "Ease of use"
    ]
}

## Prompt Template

In [19]:
# get the entities and aspects as a string
entities_aspects_str = json.dumps(entities_aspects, indent=4)

In [21]:
# an example of the output the llm should produce
output_example = {
    "Customer Service": {
        "Responsiveness": "Negative",
        "Friendliness/professionalism": "Negative",
        "Knowledge/competence": "Negative",
        "Availability (e.g., 24/7 support)": "Negative"
    },
    "Chatbots": {
        "User-friendliness": "Neutral",
        "Response accuracy": "Negative",
        "Ability to understand query": "Negative"
    }
}

output_example_str = json.dumps(output_example, indent=4)

In [57]:
# set the template string
template_string = """\
Given the following entities and aspects per entity:

```json
{entities_aspects}
```

Please perform Aspect Based Sentiment Analysis on the following text:

```text
{text}
```

Only the JSON output is expected without any "```json" text surrounding it. Do not answer with anything except JSON. Only respond with entities and aspects present in the text.

Output example:

```json
{output_example}
```
"""

In [58]:
# create the prompt template
prompt_template = ChatPromptTemplate.from_template(template_string)
prompt_template

ChatPromptTemplate(input_variables=['output_example', 'text', 'entities_aspects'], output_parser=None, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['entities_aspects', 'output_example', 'text'], output_parser=None, partial_variables={}, template='Given the following entities and aspects per entity:\n\n```json\n{entities_aspects}\n```\n\nPlease perform Aspect Based Sentiment Analysis on the following text:\n\n```text\n{text}\n```\n\nOnly the JSON output is expected without any "```json" text surrounding it. Do not answer with anything except JSON. Only respond with entities and aspects present in the text.\n\nOutput example:\n\n```json\n{output_example}\n```\n', template_format='f-string', validate_template=True), additional_kwargs={})])

### Test the template

In [64]:
# get the complaint text
review_title = df_source.loc[1, 'review_title']
review_content = df_source.loc[1, 'review_content']
complaint_text = f'# {review_title}\n\n{review_content}'

pprint(complaint_text)

('# Airtime charged, but not credited to my phone\n'
 '\n'
 'Bought Airtime online through the Vodacom App and on the last step there was '
 'an error. The money has come off my credit card, but the airtime has not '
 "been credited to my phone. Tried the online chat and was thrown out ('your "
 "chat has been ended'), then tried calling the helpline and again did not get "
 'anywhere - told there were high caller volumes and I should chat tot TOBI '
 'the chatbot, or call back later and then was thrown out of the call. No '
 'option to just stay in a waiting line.... \n'
 'Very bad service Vodacom! ')


In [65]:
# create the prompt
prompt = prompt_template.format_messages(
    entities_aspects=entities_aspects_str,
    text=complaint_text,
    output_example=output_example_str
)

#print(prompt[0].content)

## Functions

In [66]:
def classify_aspects(text:str,
                     entities_aspects:str,
                     output_example:str,
                     prompt_template:ChatPromptTemplate) -> Dict:
    """Classify the aspects of a given text and return an output dictionary."""
    chat = ChatOpenAI(
        temperature=0.0,
        max_tokens=512,
        model='gpt-4')

    # create the prompt
    prompt = prompt_template.format_messages(
        entities_aspects=entities_aspects_str,
        text=complaint_text,
        output_example=output_example_str
    )

    # get the llm response
    response = chat(prompt)

    # return the result
    return json.loads(response.content)

# test the function
pprint(classify_aspects(
    text=complaint_text,
    entities_aspects=entities_aspects_str,
    output_example=output_example_str,
    prompt_template=prompt_template
))  

{'Account & Plans': {'Account management (online portal/apps)': 'Negative'},
 'Billing & Payments': {'Refunds/credits': 'Negative'},
 'Chatbots': {'Ability to understand query': 'Negative',
              'User-friendliness': 'Negative'},
 'Customer Service': {'Availability (e.g., 24/7 support)': 'Negative',
                      'Responsiveness': 'Negative'}}


Given the following entities and aspects per entity:

```python
entities_aspects = {
    "Billing & Payments": [
        "Billing accuracy",
        "Payment methods",
        "Refunds/credits",
        "Hidden charges",
        "Monthly costs"
    ],
    "Network & Connectivity": [
        "Signal strength",
        "Network coverage",
        "Data speeds (4G, 5G, etc.)",
        "Call quality",
        "Dropped calls",
        "Roaming"
    ],
    "Customer Service": [
        "Responsiveness",
        "Friendliness/professionalism",
        "Knowledge/competence",
        "Resolution time",
        "Availability (e.g., 24/7 support)"
    ],
    "Chatbots": [
        "User-friendliness",
        "Response accuracy",
        "Speed of response",
        "Ability to understand query",
        "Escalation to human agents"
    ],
    "Account & Plans": [
        "Account management (online portal/apps)",
        "Plan flexibility",
        "Plan pricing",
        "Upgrade/downgrade process",
        "Promotions and offers"
    ],
    "Hardware/Devices": [
        "Setup/ease of installation",
        "Device reliability",
        "Device performance/speed",
        "Rental vs. purchase options",
        "Technical issues"
    ],
    "Value-added Services": [
        "Quality of service",
        "Pricing/value for money",
        "Reliability",
        "Content variety (for streaming)",
        "Ease of use"
    ]
}
```

Please perform Aspect Based Sentiment Analysis on the following text:

```text
On 12 December 2022 I took out a new 24 month contract with Vodacom for a Lenovo tablet.

The device is faulty. Its response time is extremely slow, it freezes and has to be restarted several times a day, it is frequently not responsive to touch, and often cuts off calls or the other party can no longer hear me.

I took the device to a Vodacom4U shop today, and the service consultant tested the tablet and agreed that it is unacceptably defective. However, he also informed me that I was not permitted to cancel the contract, and said the only way to remedy the situation would be to take out yet another contract with a new device, while continuing to pay for a contract that includes a device that doesn't work, until December 2024.

In terms of the Consumer Protection Act, this conduct by Vodacom is illegal. I have tried to raise the matter with your customer care line, but was directed to a bot that was unable to assist me. It was impossible to find an actual human to assist me.

In terms of applicable legislation, I have the following rights:

The supply and use of goods that are free of defects and of a quality that consumers have a right to expect. Suppliers are required to remedy defects in goods supplied or refund.

Goods of good quality, in good working order, and free of defects.

In terms of the Act, Vodacom implied a warranty of quality by making the offer, i.e. that I should have been provided with goods that are good quality. It is my right to return the defective tablet, without penalty and at Vodacom's risk and expense, within 6 months if it is of inferior quality or defective - which it is, as confirmed by your representative this morning. I then have the right to choose whether Vodacom refunds, repairs or replaces my device, and I want a complete refund.

Naturally I also want to cancel this contract due to poor quality & defects in the device you supplied, and the Act clearly states that Vodacom may not use this as grounds to enter into a new agreement (which is what your consultant to whom I spoke this morning tried to do).

I am justifiably frustrated with the poor performance of the device, the untruthful advice from Vodacom that I may not cancel the contract, and that Vodacom's customer care call center is, in fact, a bot.

As a loyal and longstanding client, I am disappointed in your service and the obstacles Vodacom has put in the way of resolving these issues. Please reply with a confirmation that this contract has been cancelled and details of my refund. I expect this matter to be resolved within 24 hours of making this complaint.
```

Only the JSON output is expected. Do not answer with anything except JSON. Only respond with entities and aspects present in the text.

Output example:

```json
{
    "Customer Service": {
        "Responsiveness": "Negative",
        "Friendliness/professionalism": "Negative",
        "Knowledge/competence": "Negative",
        "Availability (e.g., 24/7 support)": "Negative"
    },
    "Chatbots": {
        "User-friendliness": "Neutral",
        "Response accuracy": "Negative",
        "Ability to understand query": "Negative"
    }
}
```